In [87]:
import pandas as pd
import numpy as np

In [119]:
# Most PRs in 2018. Computed with December 24, 2018 db export.

results_path = "WCA_export_Results.tsv"
results_df = pd.read_table(results_path)
comps_path = "WCA_export_Competitions.tsv"
comps_df = pd.read_table(comps_path)

In [120]:
comps_2018_df = comps_df[comps_df['year'] == 2018]
comps_before_2018_df = comps_df[comps_df['year'] != 2018]

In [90]:
results_df = results_df[['competitionId', 'personId', 'eventId', 'average', 'value1', 'value2', 'value3', 'value4', 'value5']]

In [91]:
# No magic, master magic, or old mbld
results_df = results_df[(results_df['eventId'] != 'magic') & (results_df['eventId'] != 'mmagic') & (results_df['eventId'] != '333mbo')]

In [92]:
merged = comps_2018_df.merge(results_df, how='left', left_on='id', right_on='competitionId')
sorted_comps = merged.sort_values(by=['month', 'day'])
merged_before_2018 = comps_before_2018_df.merge(results_df, how='left', left_on='id', right_on='competitionId')

In [93]:
def best_avg(results):
    def helper(rows):
        times = list(filter(lambda x: x > 0, rows['average']))
        if times:
            return min(times)
        return float('inf')
    return results.groupby('eventId').apply(helper)

merged_before_2018 = merged_before_2018[['competitionId', 'personId', 'eventId', 'average', 'value1', 'value2', 'value3', 'value4', 'value5']]
pr_averages_before_2018 = merged_before_2018.groupby('personId').apply(best_avg)

In [94]:
pr_avg_before_2018_dict = pr_averages_before_2018.to_dict()

In [95]:
def best_single(results):
    times = list(filter(lambda x: x > 0, results))
    if times:
        return min(times)
    return float('inf')

def best_single_before_2017(results):
    def helper(rows):
        times = list(filter(lambda x: x > 0, rows['best_single']))
        if times:
            return min(times)
        return float('inf')
    return results.groupby('eventId').apply(helper)

merged_before_2018['singles'] = merged_before_2018[['value1', 'value2', 'value3', 'value4', 'value5']].values.tolist()
merged_before_2018['best_single'] = merged_before_2018['singles'].apply(best_single)
pr_singles_before_2018 = merged_before_2018.groupby('personId').apply(best_single_before_2017)

In [96]:
pr_singles_before_2018_dict = prs_before_2018.to_dict()

In [97]:
sorted_comps = sorted_comps[['competitionId', 'personId', 'eventId', 'average', 'value1', 'value2', 'value3', 'value4', 'value5']]

In [98]:
def count_pr_avgs(results):
    def helper(rows):
        event_id = rows['eventId'].iloc[0]
        person_id = rows['personId'].iloc[0]
        try:
            pr_avg = pr_avg_before_2018_dict[(person_id, event_id)]
        except KeyError:
            pr_avg = float('inf')
        num_prs = 0
        for avg in rows['average']:
            if avg < pr_avg and avg > 0:
                num_prs += 1
                pr_avg = avg
        return num_prs
    return results.groupby('eventId').apply(helper)

num_pr_avgs_by_person = sorted_comps.groupby('personId').apply(count_pr_avgs)

In [99]:
num_pr_avgs_by_person = num_pr_avgs_by_person.to_frame().reset_index().groupby('personId').sum().sort_values(by=0, ascending=False).reset_index()

In [100]:
sorted_comps['singles'] = sorted_comps[['value1', 'value2', 'value3', 'value4', 'value5']].values.tolist()
sorted_comps['best_single'] = sorted_comps['singles'].apply(best_single)

In [101]:
def count_pr_singles(results):
    def helper(rows):
        event_id = rows['eventId'].iloc[0]
        person_id = rows['personId'].iloc[0]
        try:
            pr_single = pr_singles_before_2018_dict[(person_id, event_id)]
        except KeyError:
            pr_single = float('inf')
        num_prs = 0
        for single in rows['best_single']:
            if single < pr_single and single > 0:
                num_prs += 1
                pr_single = single
        return num_prs
    return results.groupby('eventId').apply(helper)

num_pr_singles_by_person = sorted_comps.groupby('personId').apply(count_pr_singles)

In [102]:
num_pr_singles_by_person = num_pr_singles_by_person.to_frame().reset_index().groupby('personId').sum().sort_values(by=0, ascending=False).reset_index()

In [112]:
num_prs = num_pr_singles_by_person.merge(num_pr_avgs_by_person, how='left', left_on='personId', right_on='personId')
num_prs = num_prs.rename(columns={'0_x': '# pr singles', '0_y': '# pr averages'})
num_prs['total # prs'] = num_prs['# pr singles'] + num_prs['# pr averages']

In [118]:
num_prs = num_prs.sort_values(by='total # prs', ascending=False)
persons_path = 'WCA_export_Persons.tsv'
persons_df = pd.read_table(persons_path)
final_df = num_prs.merge(persons_df, how='left', left_on='personId', right_on='id')[['name', 'personId', '# pr singles', '# pr averages', 'total # prs']]
final_df

,name,personId,# pr singles,# pr averages,total # prs
0,Max Siauw,2017SIAU02,73,77,150
1,Hannah Minas,2017MINA04,76,67,143
2,János Bereczki,2018BERE01,64,75,139
3,Carter Kucala,2015KUCA01,64,72,136
4,Jason White,2016WHIT16,71,62,133
5,Mitchell Hum,2017HUMM01,59,70,129
6,Katie Davies,2017DAVI03,62,65,127
7,Lakshmi Sai Srivatsa,2017SRIV11,62,64,126
8,Zachary White,2010WHIT05,57,66,123
9,Sean Hartman,2016HART02,62,60,122
